# MultiSearch Rag Application

Goal: Buld an application that loads data from various sources like Arxiv, Wiki, and PDF into a LLM that generates a relevant output.

## Wiki Data Tool

In [2]:
# Searches the Wikipedia API
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun

# Wrapper for Wikipedia API to help conduct relevant searches
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

In [4]:
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=150)
wiki_wrapper

WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/druestaples/.pyenv/versions/3.8.12/envs/multisearch_rag_env/lib/python3.8/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=150)

In [5]:
# wiki_query_run = WikipediaQueryRun(name="wiki_run"
#                                    description="",
#                                    args_schema=,
#                                    return_direct=True,
#                                    verbose=True,
#                                    api_wrapper=wiki_wrapper)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)
wiki_tool

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/druestaples/.pyenv/versions/3.8.12/envs/multisearch_rag_env/lib/python3.8/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=150))

In [6]:
wiki_tool.args

{'query': {'title': 'Query',
  'description': 'query to look up on wikipedia',
  'type': 'string'}}

In [7]:
wiki_tool.json

<bound method BaseModel.json of WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/druestaples/.pyenv/versions/3.8.12/envs/multisearch_rag_env/lib/python3.8/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=150))>

In [8]:
wiki_tool.name

'wikipedia'

## Web Base Tool

In [14]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS


In [28]:
# First Data Source: Soccer Ranking Data

loader = WebBaseLoader("https://footballdatabase.com/ranking/world/1")
docs = loader.load()
docs

[Document(metadata={'source': 'https://footballdatabase.com/ranking/world/1', 'title': 'World Football / Soccer Clubs Ranking - FootballDatabase', 'description': 'Manchester City from England is the best football / soccer team of the world today. Real Madrid (Spain) comes in second place while Inter Milan (Italy) is third. Arsenal from England and Bayer Leverkusen from Germany complete the Top 5. Check out our complete club world rankings.', 'language': 'en'}, page_content="\n\n\n\n\n\nWorld Football / Soccer Clubs Ranking - FootballDatabase\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\nFootballDatabase\n\n\n\nRanking \n\nWorld Club Ranking\n\nContinental Rankings\nEurope\nSouth America\nAfrica\nAsia\nNorth America\n\n\nCompetitions \n\nShow All\n\nTop International\nUEFA Champions League\nEuropa League\nCopa Libertadores\n\nTop Domestic\nEngland Premier League\nGermany Bunde

In [16]:
# Split Documents into chunks 
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
documents

[Document(metadata={'source': 'https://footballdatabase.com/ranking/world/1', 'title': 'World Football / Soccer Clubs Ranking - FootballDatabase', 'description': 'Manchester City from England is the best football / soccer team of the world today. Real Madrid (Spain) comes in second place while Inter Milan (Italy) is third. Arsenal from England and Bayer Leverkusen from Germany complete the Top 5. Check out our complete club world rankings.', 'language': 'en'}, page_content='World Football / Soccer Clubs Ranking - FootballDatabase\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\nFootballDatabase\n\n\n\nRanking \n\nWorld Club Ranking\n\nContinental Rankings\nEurope\nSouth America\nAfrica\nAsia\nNorth America\n\n\nCompetitions \n\nShow All\n\nTop International\nUEFA Champions League\nEuropa League\nCopa Libertadores\n\nTop Domestic\nEngland Premier League\nGermany Bundesliga\nSpain

In [21]:
# Embed documents and save to vector store
vector_db = FAISS.from_documents(documents=documents, embedding=OllamaEmbeddings())
vector_db

In [22]:
# Instantiate retriever
retriever = vector_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10fb97ee0>)

In [23]:
'''
create_retrieval_tool:

Create a retriever tool (contains the configurations for retrieval) to do the retrieval of documents

1. The name of the tool
2. A description of what the tool is
3. JSON schema of what the inputs to the tool are
4. The function to call
5. Whether the result of a tool should be returned directly to the user
'''
from langchain.tools.retriever import create_retriever_tool

In [142]:
retriever_tool = create_retriever_tool(retriever=retriever,
                                       name='soccer_search',
                                       description="Search soccer rankings,how each team did to the year prior, possible future predictions, how teams can earn a higher ranking, and general information about the teams performance."
                                       )
retriever_tool

Tool(name='soccer_search', description='Search soccer rankings,how each team did to the year prior, possible future predictions, how teams can earn a higher ranking, and general information about the teams performance.', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10ad75ee0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10fb97ee0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x10ad8d040>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10fb97ee0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))

In [27]:
retriever_tool.name

'soccer_search'

## Arxiv Data Tool

In [30]:
# Tool used to do searches through Arxiv reasearch papers
from langchain_community.tools.arxiv import ArxivQueryRun
# Wrapper used to contain configuration details regarding the search tool mentioned above
from langchain_community.utilities import ArxivAPIWrapper

In [32]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_wrapper

ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)

In [33]:
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200))

In [34]:
arxiv_tool.name

'arxiv'

In [213]:
tools = [wiki_tool, arxiv_tool, retriever_tool]
# tools = [wiki_tool]
# tools = [arxiv_tool]

tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/druestaples/.pyenv/versions/3.8.12/envs/multisearch_rag_env/lib/python3.8/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=150)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='soccer_search', description='Search soccer rankings,how each team did to the year prior, possible future predictions, how teams can earn a higher ranking, and general information about the teams performance.', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10ad75ee0>, r

## LLM

In [212]:
# Create LLM
from langchain.llms import Ollama
from langchain_community.chat_models import ChatOllama
llama2 = Ollama(model="llama2")
gemma2 = Ollama(model="gemma2")
chat_ollama = ChatOllama(model="llama2")
llama2, gemma2

(Ollama(), Ollama(model='gemma2'))

## Prompts

### ChatPromptTemplate

In [174]:

# Create Prompt from Template - Doesn't work - needs variables: agent_scratchpad, input, tools, tool_names
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    template=""" 
            Answer the question or query to the best of your knowledge.
            Can you also make your answer rhyme if the result if more than six words. 
            <context>
            {context}
            </context>
            ASP: {agent_scratchpad}
            Question: {input}

            """
#     input_variables=["context", "input", "agent_scratchpad"]
#             ASP: {agent_scratchpad}

    )

In [175]:
prompt.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'context', 'input'], template=' \n            Answer the question or query to the best of your knowledge.\n            Can you also make your answer rhyme if the result if more than six words. \n            <context>\n            {context}\n            </context>\n            ASP: {agent_scratchpad}\n            Question: {input}\n\n            '))]

In [176]:
prompt.input_variables

['agent_scratchpad', 'context', 'input']

### Or download pre-made prompts from the Langchain Hub


In [177]:
# [https://smith.langchain.com/hub]

from langchain import hub

In [201]:
# premade_prompt = hub.pull("rlm/rag-prompt")
# premade_prompt = hub.pull("hwchase17/react")
# premade_prompt = hub.pull("hwchase17/openai-tools-agent")
# premade_prompt = hub.pull("chuxij/open-interpreter-system")
# premade_prompt = hub.pull("smithing-gold/assumption-checker")
premade_prompt = hub.pull("hwchase17/openai-functions-agent")

premade_prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplat

In [202]:
premade_prompt.input_variables

['agent_scratchpad', 'input']

### Prompt Template

In [203]:
from langchain.prompts.chat import MessagesPlaceholder

In [204]:
prompt3 = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant who retrieves information from documents."),
        ("user", "{question}\n{context}"),
        MessagesPlaceholder(variable_name="agent_scratchpad", optional=True),
    ]
)
prompt3

ChatPromptTemplate(input_variables=['context', 'question'], optional_variables=['agent_scratchpad'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'agent_scratchpad': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant who retrieves information from documents.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='{question}\n{context}')), MessagesPlaceholder(variable_name='agent_scratchpad', optional=True)])

Agents: background process where the LLM are used to makes the decisions on which actions to take and in which order.

## Agent

In [205]:
# Create Agents 
from langchain.agents import create_openai_tools_agent


In [214]:
# agent = create_openai_tools_agent(llm=llama2 , tools=tools, prompt=premade_prompt)
# agent = create_openai_tools_agent(llm=llama2 , tools=tools, prompt=prompt)
# agent = create_openai_tools_agent(llm=llama2 , tools=tools, prompt=prompt3)
agent = create_openai_tools_agent(llm=chat_ollama , tools=tools, prompt=premade_prompt)

agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087

In [215]:
# Execute the Agent
from langchain.agents import AgentExecutor

agent_executor=AgentExecutor(agent=agent, tools=tools ,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_comm

In [216]:
# Invoke the agent to take an action and execute some response

agent_executor.invoke({"input": "Hi"})



> Entering new AgentExecutor chain...
Hello! It's nice to meet you. Is there something I can help you with or would you like me to assist you in any way? Please let me know how I can be of service.

> Finished chain.


{'input': 'Hi',
 'output': "Hello! It's nice to meet you. Is there something I can help you with or would you like me to assist you in any way? Please let me know how I can be of service."}

In [217]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087

In [218]:
agent.get_prompts

<bound method Runnable.get_prompts of RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d1

In [219]:
agent_executor.invoke({"input": "Manchester City"})



> Entering new AgentExecutor chain...
Great, Manchester City is a popular football club based in Manchester, England! Here are some key facts and figures about the team:

1. Nickname: City
2. Founded: 1880
3. Home ground: Etihad Stadium (Capacity: 55,097)
4. Current Manager: Pep Guardiola
5. Most successful period: 2011-present (Under manager Manuel Pellegrini and Pep Guardiola, City have won several domestic and international titles, including the Premier League, FA Cup, League Cup, and Champions League)
6. Top scorer: Sergio Agüero (185 goals)
7. Most appearances: Vincent Kompany (390 apps)
8. Legendary players: Colin Bell, Francis Lee, Mike Summerbee, Tony Book, and Kevin De Bruyne
9. Rivalries: Manchester United, Liverpool, and Everton
10. Current squad: Ederson, Zack Steffen, Claudio Bravo; Kyle Walker, Aymeric Laporte, Nicolás Otamendi, John Stones, Fernandinho; Kevin De Bruyne, Bernardo Silva, David Silva, Raheem Sterling, Riyad Mahrez; Sergio Agüero, Gabriel Jesus

I hope thi

{'input': 'Manchester City',
 'output': 'Great, Manchester City is a popular football club based in Manchester, England! Here are some key facts and figures about the team:\n\n1. Nickname: City\n2. Founded: 1880\n3. Home ground: Etihad Stadium (Capacity: 55,097)\n4. Current Manager: Pep Guardiola\n5. Most successful period: 2011-present (Under manager Manuel Pellegrini and Pep Guardiola, City have won several domestic and international titles, including the Premier League, FA Cup, League Cup, and Champions League)\n6. Top scorer: Sergio Agüero (185 goals)\n7. Most appearances: Vincent Kompany (390 apps)\n8. Legendary players: Colin Bell, Francis Lee, Mike Summerbee, Tony Book, and Kevin De Bruyne\n9. Rivalries: Manchester United, Liverpool, and Everton\n10. Current squad: Ederson, Zack Steffen, Claudio Bravo; Kyle Walker, Aymeric Laporte, Nicolás Otamendi, John Stones, Fernandinho; Kevin De Bruyne, Bernardo Silva, David Silva, Raheem Sterling, Riyad Mahrez; Sergio Agüero, Gabriel Jesus

In [220]:
agent_executor.invoke({"input": "What soccer team is ranked the highest in MLS."})



> Entering new AgentExecutor chain...

Currently, the top-ranked team in Major League Soccer (MLS) is Seattle Sounders FC, who are currently ranked #1 in the MLS Power Rankings. The Sounders have had a strong season so far, with a 9-4-6 record and 33 points, leading the Western Conference. They have also shown great form in recent weeks, winning their last three matches in a row. Other top teams in MLS include Los Angeles FC, Toronto FC, and Sporting Kansas City, who are all vying for the top spot in the league.

> Finished chain.


{'input': 'What soccer team is ranked the highest in MLS.',
 'output': '\nCurrently, the top-ranked team in Major League Soccer (MLS) is Seattle Sounders FC, who are currently ranked #1 in the MLS Power Rankings. The Sounders have had a strong season so far, with a 9-4-6 record and 33 points, leading the Western Conference. They have also shown great form in recent weeks, winning their last three matches in a row. Other top teams in MLS include Los Angeles FC, Toronto FC, and Sporting Kansas City, who are all vying for the top spot in the league.'}

In [221]:
agent_executor.invoke({"input": "List the MLS teams in order from highest ranked to lowest ranked."})



> Entering new AgentExecutor chain...
Of course, I'd be happy to help! Here are the Major League Soccer (MLS) teams ranked from highest to lowest based on their current rankings:

1. Seattle Sounders FC - Currently ranked 3rd in MLS
2. Los Angeles FC - Currently ranked 2nd in MLS
3. Sporting Kansas City - Currently ranked 5th in MLS
4. New York Red Bulls - Currently ranked 6th in MLS
5. Philadelphia Union - Currently ranked 7th in MLS
6. Toronto FC - Currently ranked 8th in MLS
7. Vancouver Whitecaps FC - Currently ranked 9th in MLS
8. Houston Dynamo - Currently ranked 10th in MLS
9. D.C. United - Currently ranked 11th in MLS
10. Colorado Rapids - Currently ranked 12th in MLS
11. Chicago Fire SC - Currently ranked 13th in MLS
12. FC Dallas - Currently ranked 14th in MLS
13. San Jose Earthquakes - Currently ranked 15th in MLS
14. Real Salt Lake - Currently ranked 16th in MLS
15. Orlando City SC - Currently ranked 17th in MLS
16. Minnesota United FC - Currently ranked 18th in MLS
17. C

{'input': 'List the MLS teams in order from highest ranked to lowest ranked.',
 'output': "Of course, I'd be happy to help! Here are the Major League Soccer (MLS) teams ranked from highest to lowest based on their current rankings:\n\n1. Seattle Sounders FC - Currently ranked 3rd in MLS\n2. Los Angeles FC - Currently ranked 2nd in MLS\n3. Sporting Kansas City - Currently ranked 5th in MLS\n4. New York Red Bulls - Currently ranked 6th in MLS\n5. Philadelphia Union - Currently ranked 7th in MLS\n6. Toronto FC - Currently ranked 8th in MLS\n7. Vancouver Whitecaps FC - Currently ranked 9th in MLS\n8. Houston Dynamo - Currently ranked 10th in MLS\n9. D.C. United - Currently ranked 11th in MLS\n10. Colorado Rapids - Currently ranked 12th in MLS\n11. Chicago Fire SC - Currently ranked 13th in MLS\n12. FC Dallas - Currently ranked 14th in MLS\n13. San Jose Earthquakes - Currently ranked 15th in MLS\n14. Real Salt Lake - Currently ranked 16th in MLS\n15. Orlando City SC - Currently ranked 17th 

In [223]:
agent_executor.invoke({"input": "Rank the top 5 MLS teams that have the most impresive trophies."})



> Entering new AgentExecutor chain...
Of course, I'd be happy to help! Here are the top 5 MLS teams with the most impressive trophy count:

1. Los Angeles Galaxy - With 21 major trophies, including 5 MLS Cups, 4 Supporters' Shields, and 3 U.S. Open Cups, the LA Galaxy are the clear leaders in terms of trophy count among MLS teams.
2. Seattle Sounders FC - With 16 major trophies, including 3 MLS Cups, 4 Supporters' Shields, and 1 U.S. Open Cup, Seattle Sounders FC is a close second to the LA Galaxy in terms of trophy count.
3. Sporting Kansas City - With 13 major trophies, including 2 MLS Cups, 3 Supporters' Shields, and 1 U.S. Open Cup, Sporting Kansas City is third on the list of teams with the most impressive trophy count in MLS.
4. New York Red Bulls - With 12 major trophies, including 1 MLS Cup, 3 Supporters' Shields, and 1 U.S. Open Cup, the New York Red Bulls are fourth on the list of teams with the most impressive trophy count in MLS.
5. Houston Dynamo - With 10 major trophies

{'input': 'Rank the top 5 MLS teams that have the most impresive trophies.',
 'output': "Of course, I'd be happy to help! Here are the top 5 MLS teams with the most impressive trophy count:\n\n1. Los Angeles Galaxy - With 21 major trophies, including 5 MLS Cups, 4 Supporters' Shields, and 3 U.S. Open Cups, the LA Galaxy are the clear leaders in terms of trophy count among MLS teams.\n2. Seattle Sounders FC - With 16 major trophies, including 3 MLS Cups, 4 Supporters' Shields, and 1 U.S. Open Cup, Seattle Sounders FC is a close second to the LA Galaxy in terms of trophy count.\n3. Sporting Kansas City - With 13 major trophies, including 2 MLS Cups, 3 Supporters' Shields, and 1 U.S. Open Cup, Sporting Kansas City is third on the list of teams with the most impressive trophy count in MLS.\n4. New York Red Bulls - With 12 major trophies, including 1 MLS Cup, 3 Supporters' Shields, and 1 U.S. Open Cup, the New York Red Bulls are fourth on the list of teams with the most impressive trophy co